# Лекція 4: Функції, Модулі, Помилки

**Курс**: Прикладна розробка програмного забезпечення (Python)
**Рік**: 2026

---

## Цілі навчання

Після цієї лекції ви зможете:

1. Використовувати **lambda-вирази**, **map/filter/reduce**, **генератори** та **декоратори** для написання лаконічного та ефективного коду
2. Розуміти **область видимості (scope)** змінних за правилом LEGB та створювати **замикання (closures)**
3. Організовувати код у **модулі та пакети**, використовувати стандартну бібліотеку Python
4. Обробляти **винятки (exceptions)** за допомогою `try`/`except`/`else`/`finally`, створювати власні винятки та застосовувати базове логування

## Передумови

Перед початком цієї лекції переконайтеся, що ви:

- Завершили **Лекцію 1** (Вступ до Python), **Лекцію 2** (Механіка мови) та **Лекцію 3** (Структури даних та Функції)
- Маєте встановлений **Python 3.13+**
- Розумієте базові типи даних, колекції (list, dict, set, tuple), comprehensions
- Вмієте створювати функції з `def`, параметрами, `*args`/`**kwargs` та значеннями за замовчуванням
- Вмієте працювати з Jupyter Notebook

## Вступ

У Лекції 3 ми навчились працювати з колекціями, comprehensions та створювати базові функції. Тепер пора піти далі — від простих функцій до потужних інструментів функціонального програмування.

**Що нового в Лекції 4:**
- Функції як об'єкти першого класу: lambda, передача функцій як аргументів, map/filter/reduce
- Генератори та ітератори — ефективна робота з великими даними
- Область видимості (LEGB) та замикання — розуміння "де живуть змінні"
- Модулі та пакети — організація коду для масштабних проєктів
- Обробка помилок — як писати надійний код

![meme](https://preview.redd.it/just-write-a-function-bro-v0-yj6ypwxwsgva1.png?width=640&crop=smart&auto=webp&s=a0f9947307cfafe1658b90a33d0218d0e6ecdbc0)

# 1. Функції (продовження)

У Лекції 3 ми познайомились з базовим синтаксисом `def`, параметрами, `return`, `*args` та `**kwargs`. Тепер зануримось у глибші концепції функціонального програмування в Python.

---

## 1.1 Лямбда-вирази (Lambda expressions)

**Лямбда-вирази** — це анонімні (безіменні) функції, що складаються з одного виразу.

**Синтаксис**: `lambda аргументи: вираз`

![Lambda Syntax Diagram](assets/diagrams/02-lambda-syntax-diagram.webp)

*Джерело: [GeeksforGeeks — Python Lambda Functions](https://www.geeksforgeeks.org/python-lambda-anonymous-functions-filter-map-reduce/)*

**Коли використовувати lambda:**
- Як короткий inline-callback (наприклад, ключ сортування)
- Коли функція настільки проста, що `def` — надмірність

**Коли НЕ використовувати:**
- Якщо логіка складна — використовуйте `def`
- Якщо функція потрібна в кількох місцях — дайте їй ім'я

In [22]:
# Lambda vs def — порівняння
# def-версія
def square_def(x):
    return x ** 2

# lambda-версія
square_lambda = lambda x: x ** 2

print(f"def:    square_def(5)    = {square_def(5)}")
print(f"lambda: square_lambda(5) = {square_lambda(5)}")

# Lambda з кількома аргументами
multiply = lambda a, b: a * b
print(f"\nmultiply(3, 7) = {multiply(3, 7)}")

def:    square_def(5)    = 25
lambda: square_lambda(5) = 25

multiply(3, 7) = 21


In [23]:
# Lambda як аргумент sorted() — найпоширеніше використання
students = [
    {"name": "Олена", "grade": 95},
    {"name": "Ігор", "grade": 87},
    {"name": "Марія", "grade": 92},
    {"name": "Андрій", "grade": 78},
]

# Сортування за оцінкою
by_grade = sorted(students, key=lambda s: s["grade"], reverse=True)
for s in by_grade:
    print(f"  {s['name']}: {s['grade']}")

  Олена: 95
  Марія: 92
  Ігор: 87
  Андрій: 78


---

## 1.2 Функції як параметри та сортування з `key`

У Python функції — це **об'єкти першого класу (first-class objects)**. Це означає, що функції можна:
- Зберігати у змінних
- Передавати як аргументи іншим функціям
- Повертати з інших функцій

### Сортування з параметром `key`

`sorted()` та `.sort()` приймають параметр `key` — функцію, що визначає, **за яким критерієм** сортувати.

```
sorted(iterable, key=функція, reverse=False)
list.sort(key=функція, reverse=False)
```

In [26]:
# Передача функції як аргументу
def apply_operation(func, value):
    """Застосовує функцію func до value."""
    return func(value)

def double(x):
    return x * 2

def negate(x):
    return -x

print(f"apply_operation(double, 5) = {apply_operation(double, 5)}")
print(f"apply_operation(negate, 5) = {apply_operation(negate, 5)}")
print(f"apply_operation(lambda x: x ** 3, 5) = {apply_operation(lambda x: x ** 3, 5)}")

apply_operation(double, 5) = 10
apply_operation(negate, 5) = -5
apply_operation(lambda x: x ** 3, 5) = 125


In [27]:
# Сортування з key — різні критерії
words = ["banana", "apple", "cherry", "date"]

# За довжиною
print(f"За довжиною:  {sorted(words, key=len)}")

# За останньою літерою
print(f"За останньою: {sorted(words, key=lambda w: w[-1])}")

# Складніший приклад: список кортежів
employees = [("Олена", "QA", 55000), ("Ігор", "Dev", 70000), ("Марія", "Dev", 65000)]

# За зарплатою (третій елемент)
by_salary = sorted(employees, key=lambda e: e[2], reverse=True)
for name, role, salary in by_salary:
    print(f"  {name} ({role}): ${salary}")

За довжиною:  ['date', 'apple', 'banana', 'cherry']
За останньою: ['banana', 'apple', 'date', 'cherry']
  Ігор (Dev): $70000
  Марія (Dev): $65000
  Олена (QA): $55000


---

## 1.3 `map`, `filter`, `reduce`

Три класичних функції функціонального програмування:

| Функція | Що робить | Повертає |
|---------|-----------|----------|
| `map(func, iterable)` | Застосовує `func` до кожного елемента | ітератор результатів |
| `filter(func, iterable)` | Залишає елементи, де `func` повертає `True` | ітератор відфільтрованих |
| `reduce(func, iterable)` | Накопичує результат, згортаючи до одного значення | одне значення |

### map/filter vs comprehensions

```python
# map + lambda
list(map(lambda x: x**2, numbers))

# Еквівалент comprehension (зазвичай краще читається)
[x**2 for x in numbers]
```

> **Порада**: Використовуйте comprehensions для простих випадків, map/filter — коли вже маєте готову функцію. Також завжди пам'ятайте, що comprehencions швидші

In [31]:
# map — застосувати функцію до кожного елемента
names = ["олена", "ігор", "марія", "андрій"]
upper_names = list(map(str.upper, names))
print(f"map(str.upper): {upper_names}")

# Довжини слів
lengths = list(map(len, names))
print(f"map(len):       {lengths}")

# Еквівалент comprehension
upper_comp = [name.upper() for name in names]
print(f"comprehension:  {upper_comp}")

# передача кількох ітерованих об'єктів до map, тут comprehension не допоможе
list_demo_1 = [1, 2, 3, 4, 5]
list_demo_2 = [3, 4, 5, 6, 7]
new_list = map(lambda x, y: x + y, list_demo_1, list_demo_2)
print(list(new_list))

map(str.upper): ['ОЛЕНА', 'ІГОР', 'МАРІЯ', 'АНДРІЙ']
map(len):       [5, 4, 5, 6]
comprehension:  ['ОЛЕНА', 'ІГОР', 'МАРІЯ', 'АНДРІЙ']
[4, 6, 8, 10, 12]


In [32]:
# filter — залишити елементи за умовою
numbers = [-3, -1, 0, 2, 4, 7, -5, 8]

positive = list(filter(lambda x: x > 0, numbers))
print(f"filter (додатні): {positive}")

even = list(filter(lambda x: x % 2 == 0, numbers))
print(f"filter (парні):   {even}")

# Еквівалент comprehension
positive_comp = [x for x in numbers if x > 0]
print(f"comprehension:    {positive_comp}")

filter (додатні): [2, 4, 7, 8]
filter (парні):   [0, 2, 4, 8]
comprehension:    [2, 4, 7, 8]


In [33]:
%%timeit
# порівняння швидкості виконання filter vs comprehension
numbers = [-1, 4, 5, -2, 0, 3, -6, 8, 7, -9] * 100000
filter_result = list(filter(lambda x: x > 0, numbers))

146 ms ± 16.4 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [34]:
%%timeit
# порівняння швидкості виконання filter vs comprehension
numbers = [-1, 4, 5, -2, 0, 3, -6, 8, 7, -9] * 100000
filter_result = [x for x in numbers if x > 0]

68.8 ms ± 5.98 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [36]:
# reduce — згортання до одного значення
from functools import reduce

numbers = [1, 2, 3, 4, 5]

# Добуток усіх чисел: 1 * 2 * 3 * 4 * 5
product = reduce(lambda a, b: a * b, numbers)
print(f"reduce (добуток): {product}")

# Як це працює покроково:
# Крок 1: a=1, b=2 → 2
# Крок 2: a=2, b=3 → 6
# Крок 3: a=6, b=4 → 24
# Крок 4: a=24, b=5 → 120

# Конкатенація рядків
words = ["Python", " ", "is", " ", "awesome"]
sentence = reduce(lambda a, b: a + b, words)
print(f"reduce (рядки):   '{sentence}'")

reduce (добуток): 120
reduce (рядки):   'Python is awesome'


---

## 1.4 Ітератори та генератори (Iterators and generators)

### Протокол ітератора (Iterator protocol)

Будь-який об'єкт, що реалізує методи `__iter__()` і `__next__()`, є **ітератором**. Саме цей протокол стоїть за `for` циклом.

```
for item in collection:  →  ітератор = iter(collection)
    ...                      next(ітератор) → елемент
                             ...
                             StopIteration → кінець
```

### Генератори (Generators)

**Генератор** — це функція, що використовує `yield` замість `return`. Вона **ліниво** (lazily) генерує значення по одному, не зберігаючи все в пам'яті.

In [37]:
# Ручна ітерація: iter() та next()
fruits = ["яблуко", "банан", "вишня"]
iterator = iter(fruits)

print(f"next(): {next(iterator)}")
print(f"next(): {next(iterator)}")
print(f"next(): {next(iterator)}")

# Наступний виклик — StopIteration
try:
    next(iterator)
except StopIteration:
    print("StopIteration — елементи закінчились!")

# range() — «ледачий» ітератор (не зберігає всі числа в пам'яті)
r = range(1_000_000)
print(f"\nrange(1_000_000) займає лише {r.__sizeof__()} байт")

next(): яблуко
next(): банан
next(): вишня
StopIteration — елементи закінчились!

range(1_000_000) займає лише 48 байт


In [40]:
# Генератор-функція з yield
def countdown(n):
    """Зворотний відлік від n до 1."""
    while n > 0:
        yield n
        n -= 1

# Використання генератора
for num in countdown(5):
    print(num, end=" ")
print()

# Генератор Фібоначчі — нескінченний!
def fibonacci():
    a, b = 0, 1
    while True:
        yield a
        a, b = b, a + b

# Беремо перші 10 чисел Фібоначчі
fib = fibonacci()
first_10 = [next(fib) for _ in range(10)]
print(f"Фібоначчі: {first_10}")

5 4 3 2 1 
Фібоначчі: [0, 1, 1, 2, 3, 5, 8, 13, 21, 34]


In [41]:
# Генераторний вираз vs list comprehension — порівняння пам'яті
import sys

# List comprehension — зберігає ВСЕ в пам'яті
list_comp = [x ** 2 for x in range(10_000)]

# Generator expression — генерує по одному
gen_expr = (x ** 2 for x in range(10_000))

print(f"List comprehension: {sys.getsizeof(list_comp):>8} байт")
print(f"Generator expression: {sys.getsizeof(gen_expr):>6} байт")
print(f"\nРізниця: list у {sys.getsizeof(list_comp) // sys.getsizeof(gen_expr)}x більший!")

# Ланцюжок генераторів (chaining)
numbers = range(100)
evens = (x for x in numbers if x % 2 == 0)
squares = (x ** 2 for x in evens)
big_squares = (x for x in squares if x > 100)

print(f"\nКвадрати парних > 100: {list(big_squares)[:10]}...")

List comprehension:    85176 байт
Generator expression:    200 байт

Різниця: list у 425x більший!

Квадрати парних > 100: [144, 196, 256, 324, 400, 484, 576, 676, 784, 900]...


Генератор — це зручний спосіб створити ітератор через yield (або вираз ( ... for ... )).
 - Кожен генератор є ітератором
 - **Не кожен** ітератор є генератором (ітератор можна написати як клас з __iter__/__next__)

In [42]:
# короткий приклад, ітератор, що не є генератором

class CountUpToIter:
    def __init__(self, n):
        self.n = n
        self.i = 0

    def __iter__(self):
        return self

    def __next__(self):
        if self.i >= self.n:
            raise StopIteration
        val = self.i
        self.i += 1
        return val

counter = CountUpToIter(5)
one = next(counter)
two = next(counter)
print(f"Ітератор CountUpToIter: {one}, {two}")

Ітератор CountUpToIter: 0, 1


Просте правило вибору:
- Генератор: “просто віддай значення потоком” (ETL, файли, батчі, фільтри)
- Ітератор-об’єкт: Єпотрібен контроль/методи/стан/метрики/ресурсиЄ (reset, seek, stats, API paginator, window with peek)

---

## 1.5 Область видимості: правило LEGB та замикання (closures)

Python шукає змінні за правилом **LEGB**:

| Рівень | Назва | Опис |
|--------|-------|------|
| **L** | Local | Змінні всередині поточної функції |
| **E** | Enclosing | Змінні в зовнішній (обгортковій) функції |
| **G** | Global | Змінні на рівні модуля |
| **B** | Built-in | Вбудовані імена Python (`print`, `len`, `range`...) |

![LEGB Rule](assets/diagrams/10-legb-rule.png)

*Джерело: [GeeksforGeeks — Scope Resolution LEGB Rule](https://www.geeksforgeeks.org/scope-resolution-in-python-legb-rule/)*

## 

### Замикання (Closures)

**Замикання** — це функція, що «пам'ятає» змінні з зовнішньої (enclosing) області видимості, навіть після завершення зовнішньої функції.

**Замикання (Closure) — як це працює:**

```
def outer(x):          # Зовнішня функція
    def inner(y):      # Внутрішня функція
        return x + y   # x "захоплена" з outer
    return inner       # Повертаємо inner

add5 = outer(5)        # x=5 збережено в замиканні
add5(3)  # -> 8        # inner(3): x=5, y=3
add5(7)  # -> 12       # inner(7): x=5, y=7
```

| Етап | Що відбувається |
|------|-----------------|
| `outer(5)` | Створює `inner`, `x=5` зберігається |
| `add5 = outer(5)` | `add5` — це `inner` із замиканням `x=5` |
| `add5(3)` | Викликає `inner(y=3)`, використовує `x=5` |

In [47]:
if 10 > 5:
    o = 1

print(o)

1


In [48]:
# Демонстрація LEGB
x = "Global"

def outer():
    x = "Enclosing"

    def inner():
        x = "Local"
        print(f"inner(): x = {x}")       # Local

    inner()
    print(f"outer(): x = {x}")           # Enclosing

outer()
print(f"module:  x = {x}")              # Global
print(f"len — Built-in: {len}")          # Built-in

inner(): x = Local
outer(): x = Enclosing
module:  x = Global
len — Built-in: <built-in function len>


In [5]:
x = 'Vasa'

def get_counter():
    n = 0

    def counter():
        nonlocal n
        x += 1
        print(f"Global x: {x}")  # Vasa
        n += 1
        return n
    
    return counter

counter = get_counter()
try:
    print(counter())
except UnboundLocalError as e:
    print(f"Помилка: {e}")
    print("Проблема в тому, що ми намагаємося змінити 'x' всередині counter(), але 'x' не визначено як nonlocal або global.")

Помилка: cannot access local variable 'x' where it is not associated with a value
Проблема в тому, що ми намагаємося змінити 'x' всередині counter(), але 'x' не визначено як nonlocal або global.


**global і nonlocal** потрібні, коли ти хочеш змінити (переприсвоїти) змінну, яка оголошена не в поточній функції.
- global: працює з глобальною змінною модуля (з рівня файлу).
- nonlocal: працює зі змінною з enclosing scope — тобто з зовнішньої функції (але не глобальної).

In [64]:
# global
x = 10

def inc():
    global x
    x += 1

inc()
print(x)  # 11

11


In [65]:
# nonlocal

def outer():
    count = 0

    def inner():
        nonlocal count
        count += 1
        return count

    return inner

f = outer()
print(f())  # 1
print(f())  # 2

1
2


**Важливий нюанс: "міняти" mutable можна без nonlocal/global**

In [69]:
def outer():
    items = []

    def inner(x):
        items.append(x)   # не переприсвоюємо items
        return items

    return inner

f = outer()
print(f(1))
print(f(2))

[1]
[1, 2]


In [71]:
def one(a):
    def two(b):
        return a + b
    
    return two

In [74]:
two = one(1)
two(2)  # 3

3

In [76]:
list(range(0))

[]

In [77]:
# Замикання (Closure) — функція "пам'ятає" зовнішню змінну
def make_multiplier(n):
    """Повертає функцію, що множить на n."""
    def multiplier(x):
        return x * n   # n "запам'ятовується" з зовнішньої функції
    return multiplier

double = make_multiplier(2)
triple = make_multiplier(3)

print(f"double(5) = {double(5)}")   # 10
print(f"triple(5) = {triple(5)}")   # 15

# Практичний приклад: лічильник
def make_counter(start=0):
    count = list(range(start))
    def counter():
        count.append(1)
        return len(count)
    return counter

my_counter = make_counter()
print(f"\ncounter: {my_counter()}, {my_counter()}, {my_counter()}")

double(5) = 10
triple(5) = 15

counter: 1, 2, 3


---

## 1.6 Декоратори — вступ

**Декоратор** — це функція, що «обгортає» іншу функцію, додаючи додаткову поведінку.

```python
@decorator
def func():
    ...

# Це те саме, що:
func = decorator(func)
```

![Decorator Pattern](assets/diagrams/13-decorator-pattern.webp)

*Джерело: [GeeksforGeeks — Decorators in Python](https://www.geeksforgeeks.org/decorators-in-python/)*

**Навіщо вони потрібні**

Декоратори дозволяють додавати функціональність без повторювання коду:
- логування
- вимірювання часу
- кешування
- retry
- перевірка прав доступу
- валідація аргументів
- контроль транзакцій / ресурсів

In [79]:
# Декоратор вручну
def trace(fn):
    def wrapper(*args, **kwargs):
        print("До виклику")
        result = fn(*args, **kwargs)
        print("Після виклику")
        return result
    return wrapper

def hello(name):
    return f"Hi, {name}!"

hello = trace(hello)      # вручну "задекорували"
print(hello("Bohdan"))

До виклику
Після виклику
Hi, Bohdan!


In [80]:
# Спосіб 2: синтаксис @decorator (те саме, але елегантніше)
def trace(fn):
    def wrapper(*args, **kwargs):
        print("До виклику")
        result = fn(*args, **kwargs)
        print("Після виклику")
        return result
    return wrapper

@trace
def hello(name):
    return f"Hi, {name}!"

Чому потрібен **functools.wraps**

Без wraps декорована функція "втрачає паспорт":
- __name__, __doc__, інколи сигнатуру (для інструментів)

Правильний стиль:

In [16]:
import functools

def trace(fn):
    @functools.wraps(fn)
    def wrapper(*args, **kwargs):
        print(f"Calling {fn.__name__}")
        return fn(*args, **kwargs)
    return wrapper

**Порядок декораторів**

```
@A
@B
def f(): ...
```
те саме, що
```
f = A(B(f))
```
Тобто B ближче до функції (внутрішній), A — зовнішній.

In [81]:
import functools

def A(fn):
    @functools.wraps(fn)
    def w(*a, **k):
        print("A before")
        r = fn(*a, **k)
        print("A after")
        return r
    return w

def B(fn):
    @functools.wraps(fn)
    def w(*a, **k):
        print("B before")
        r = fn(*a, **k)
        print("B after")
        return r
    return w

@A
@B
def f():
    print("f body")

f()

A before
B before
f body
B after
A after


**Декоратор з параметрами (коли хочеш ```@retry(times=3)```)**

```
@retry(times=3)
def f(): ...
```
це те саме, що
```
f = retry(times=3)(f)

In [18]:
import functools, time

def retry(times=3, delay=0.2, exceptions=(Exception,)):
    def decorate(fn):
        @functools.wraps(fn)
        def wrapper(*args, **kwargs):
            last_exc = None
            for attempt in range(1, times + 1):
                try:
                    return fn(*args, **kwargs)
                except exceptions as e:
                    last_exc = e
                    if attempt < times:
                        time.sleep(delay)
            raise last_exc
        return wrapper
    return decorate

@retry(times=5, delay=0.1)
def fragile_call():
    ...

---

## 1.7 Підказки типів (Type hints intro)

Python — динамічно типізована мова, але з версії 3.5+ підтримує **підказки типів (type hints)** для покращення читабельності та підтримки інструментів.

**Навіщо type hints:**
- Покращують читабельність коду
- IDE підказує методи та помилки
- Інструменти типу  знаходять баги до запуску

**Важливо**: Type hints **НЕ впливають** на виконання — Python їх ігнорує під час роботи.

In [82]:
# Базові type hints
name: str = "Bohdan"
def greet(name: str) -> str:
    return f"Привіт, {name}!"

def add(a: int, b: int) -> int:
    return a + b

print(greet("Олена"))
print(add(3, 7))

# Type hints НЕ блокують "неправильні" типи
print(add("hello", " world"))  # Працює! Python не перевіряє типи під час виконання

Привіт, Олена!
10
hello world


In [ ]:
# Складніші типи
from typing import List, Dict, Optional

def find_student(students: List[Dict], name: str) -> Optional[Dict]:
    """Шукає студента за іменем. Повертає None, якщо не знайдено."""
    for s in students:
        if s["name"] == name:
            return s
    return None

# Python 3.10+ синтаксис: X | None замість Optional[X]
def divide(a: float, b: float) -> float | None:
    if b == 0:
        return None
    return a / b

# Анотації змінних
students:  [Dict[str, str | int]] = [
    {"name": "Олена", "grade": 95},
    {"name": "Ігор", "grade": 87},
]

result = find_student(students, "Олена")
print(f"Знайдено: {result}")
print(f"divide(10, 3) = {divide(10, 3):.2f}")
print(f"divide(10, 0) = {divide(10, 0)}")

Знайдено: {'name': 'Олена', 'grade': 95}
divide(10, 3) = 3.33
divide(10, 0) = None


---

# 2. Модулі та імпорти (Modules & Imports)

---

## 2.1 Механізми імпорту (Import mechanisms)

Python організовує код у **модулі** — файли `.py`, які можна імпортувати.

| Синтаксис | Що імпортує | Приклад |
|-----------|------------|---------|
| `import module` | Весь модуль | `import math` |
| `from module import name` | Конкретне ім'я | `from math import sqrt` |
| `import module as alias` | З псевдонімом | `import datetime as dt` |
| `from module import *` | Все (уникайте!) | `from math import *` |

**PEP 8 порядок імпортів** (розділяти порожнім рядком):
1. Стандартна бібліотека
2. Сторонні пакети
3. Локальні модулі

**Механізми імпорту в Python:**

```
import math              # Імпорт всього модуля
import math as m         # Імпорт з псевдонімом
from math import sqrt    # Імпорт конкретної функції
from math import *       # Імпорт всього (НЕ рекомендовано!)
```

| Стиль | Використання | Рекомендація |
|-------|-------------|--------------|
| `import module` | `module.func()` | Найкращий для великих модулів |
| `import module as alias` | `alias.func()` | Для довгих імен |
| `from module import func` | `func()` | Для часто використовуваних |
| `from module import *` | `func()` | Уникайте! |

**Порядок імпортів (PEP 8):**

```python
# 1. Стандартна бібліотека
import os
import sys
from datetime import datetime

# 2. Сторонні бібліотеки
import requests
import numpy as np

# 3. Локальні модулі
from mypackage import utils
from .helpers import validate
```

In [87]:
from os import listdir
listdir(".")

['assets', 'lecture-04.ipynb']

In [6]:
# Різні стилі імпорту
import math
from datetime import datetime, timedelta
import random as rnd
from collections import Counter

# import math — доступ через крапку
print(f"math.pi = {math.pi:.4f}")
print(f"math.sqrt(144) = {math.sqrt(144)}")

# from ... import — прямий доступ
now = datetime.now()
print(f"\nЗараз: {now.strftime('%Y-%m-%d %H:%M')}")

# alias
print(f"\nВипадкове число: {rnd.randint(1, 100)}")

math.pi = 3.1416
math.sqrt(144) = 12.0

Зараз: 2026-02-26 10:29

Випадкове число: 97


In [7]:
# Чому from module import * — погана практика?
# from math import *   # Імпортує sin, cos, pi, e, log... все в namespace!
# from os import *      # path, getcwd, listdir... КОНФЛІКТ з math?

# Кращий підхід: імпортувати конкретні імена
from math import pi, sqrt, ceil
from os.path import exists, join

print(f"pi = {pi}")
print(f"sqrt(16) = {sqrt(16)}")
print(f"ceil(3.2) = {ceil(3.2)}")

pi = 3.141592653589793
sqrt(16) = 4.0
ceil(3.2) = 4


---

## 2.2 Огляд стандартної бібліотеки (Standard library overview)

Python поставляється з багатою стандартною бібліотекою — "batteries included":

| Модуль | Призначення |
|--------|------------|
| `math` | Математичні функції |
| `datetime` | Дати та час |
| `random` | Випадкові числа |
| `json` | Робота з JSON |
| `os` / `pathlib` | Файлова система |
| `sys` | Системна інформація |
| `collections` | Розширені колекції |

**Ключові модулі стандартної бібліотеки:**

| Категорія | Модулі | Призначення |
|-----------|--------|-------------|
| Файли/OS | `os`, `pathlib`, `shutil` | Робота з файловою системою |
| Дані | `json`, `csv`, `sqlite3` | Серіалізація та БД |
| Текст | `re`, `string`, `textwrap` | Обробка тексту |
| Час | `datetime`, `time`, `calendar` | Дата та час |
| Математика | `math`, `random`, `statistics` | Обчислення |
| Колекції | `collections`, `itertools` | Розширені структури |
| Мережа | `urllib`, `http`, `socket` | HTTP та мережа |
| Тестування | `unittest`, `doctest` | Тестування коду |

**Як знайти потрібний модуль:**

```python
# Перегляд всіх модулів
help('modules')

# Перегляд вмісту модуля
import os
dir(os)           # Список всіх імен
help(os.path)     # Документація
```


In [8]:
# Демо ключових модулів стандартної бібліотеки
import math
import datetime
import random
import json

# math
print("=== math ===")
print(f"  pi = {math.pi:.6f}, e = {math.e:.6f}")
print(f"  factorial(10) = {math.factorial(10)}")

# datetime
print("\n=== datetime ===")
now = datetime.datetime.now()
birthday = datetime.date(2000, 5, 15)
age = (datetime.date.today() - birthday).days // 365
print(f"  Зараз: {now.strftime('%d.%m.%Y %H:%M')}")
print(f"  Вік: {age} років")

# random
print("\n=== random ===")
print(f"  randint(1, 6): {random.randint(1, 6)} (кидок кубика)")
print(f"  choice: {random.choice(['Python', 'Java', 'Rust'])}")

# json
print("\n=== json ===")
data = {"name": "Олена", "grade": 95}
json_str = json.dumps(data, ensure_ascii=False, indent=2)
print(f"  JSON: {json_str}")

=== math ===
  pi = 3.141593, e = 2.718282
  factorial(10) = 3628800

=== datetime ===
  Зараз: 26.02.2026 10:29
  Вік: 25 років

=== random ===
  randint(1, 6): 6 (кидок кубика)
  choice: Python

=== json ===
  JSON: {
  "name": "Олена",
  "grade": 95
}


In [9]:
# collections — розширені колекції
from collections import Counter, defaultdict, namedtuple

# Counter — підрахунок елементів
text = "абракадабра"
letter_count = Counter(text)
print(f"Counter: {letter_count}")
print(f"Топ-3: {letter_count.most_common(3)}")

# defaultdict — словник зі значенням за замовчуванням
groups = defaultdict(list)
students = [("math", "Олена"), ("physics", "Ігор"), ("math", "Марія"), ("physics", "Андрій")]
for subject, name in students:
    groups[subject].append(name)
print(f"\nГрупи: {dict(groups)}")

# namedtuple — кортеж з іменованими полями (попередній перегляд)
Point = namedtuple("Point", ["x", "y"])
p = Point(10, 20)
print(f"\nPoint: {p}, x={p.x}, y={p.y}")

Counter: Counter({'а': 5, 'б': 2, 'р': 2, 'к': 1, 'д': 1})
Топ-3: [('а', 5), ('б', 2), ('р', 2)]

Групи: {'math': ['Олена', 'Марія'], 'physics': ['Ігор', 'Андрій']}

Point: Point(x=10, y=20), x=10, y=20


---

## 2.3 Створення власних модулів + `__name__`

Будь-який файл `.py` — це модуль, який можна імпортувати.

### Патерн `if __name__ == "__main__":`

Коли Python запускає файл напряму: `__name__ == "__main__"`
Коли файл імпортується: `__name__ == "ім'я_модуля"`

**Як працює `__name__`:**

```
┌─────────────────────────────────────────┐
│  my_module.py                           │
│                                         │
│  def greet():                           │
│      print("Hello!")                    │
│                                         │
│  if __name__ == "__main__":             │
│      greet()  # Виконується тільки      │
│               # при прямому запуску     │
└─────────────────────────────────────────┘

Запуск напряму:     python my_module.py  → __name__ == "__main__" ✓
Імпорт з іншого:    import my_module     → __name__ == "my_module" ✗
```

**Потік виконання при імпорті:**

| Сценарій | `__name__` | `if __name__ == "__main__":` |
|----------|-----------|------------------------------|
| `python script.py` | `"__main__"` | Виконується |
| `import script` | `"script"` | НЕ виконується |
| `from script import func` | `"script"` | НЕ виконується |

In [10]:
# Симуляція створення модуля (в Jupyter ми покажемо концепцію)
# Уявіть файл validators.py:

# === validators.py ===
def validate_email(email: str) -> bool:
    """Перевіряє, чи email має правильний формат."""
    return "@" in email and "." in email.split("@")[-1]

def validate_age(age: int) -> bool:
    """Перевіряє, чи вік у допустимому діапазоні."""
    return 0 < age < 150

def validate_password(password: str) -> bool:
    """Перевіряє мінімальну довжину пароля."""
    return len(password) >= 8

# if __name__ == "__main__":
#     # Цей код виконується ТІЛЬКИ при прямому запуску
#     print(validate_email("test@example.com"))  # True
#     print(validate_age(25))                     # True

# Демонстрація
print(f"validate_email('test@example.com'): {validate_email('test@example.com')}")
print(f"validate_email('invalid'):          {validate_email('invalid')}")
print(f"validate_age(25):                   {validate_age(25)}")
print(f"validate_password('short'):         {validate_password('short')}")
print(f"validate_password('long_enough_password'): {validate_password('long_enough_password')}")

validate_email('test@example.com'): True
validate_email('invalid'):          False
validate_age(25):                   True
validate_password('short'):         False
validate_password('long_enough_password'): True


---

## 2.4 Основи структури пакетів (Package structure)

**Пакет (package)** — це директорія з файлом `__init__.py`, що містить модулі.

```text
my_project/
├── main.py
├── utils/
│   ├── __init__.py          ← робить utils пакетом
│   ├── validators.py
│   └── formatters.py
└── models/
    ├── __init__.py
    └── student.py
```

```python
# Імпорт з пакету
from utils.validators import validate_email
from models.student import Student
```

**Структура пакету Python:**

```
my_package/
├── __init__.py      # Робить директорію пакетом
├── module_a.py      # Підмодуль A
├── module_b.py      # Підмодуль B
└── subpackage/      # Вкладений пакет
    ├── __init__.py
    └── module_c.py
```

**Роль `__init__.py`:**

| Функція | Приклад |
|---------|---------|
| Робить директорію пакетом | Порожній файл — мінімум |
| Реекспорт імен | `from .module_a import func` |
| Ініціалізація пакету | Код виконується при імпорті |
| Визначення `__all__` | Контролює `from package import *` |


> **Примітка**: Починаючи з Python 3.3, `__init__.py` не є обов'язковим (namespace packages), але його рекомендується створювати для явності.

In [11]:
# Приклад структури проєкту (показуємо через print)
project_structure = """
student_manager/
├── main.py                  # Точка входу
├── data/
│   ├── __init__.py          # from data import load_students
│   └── loader.py            # Завантаження даних
├── services/
│   ├── __init__.py
│   └── grade_service.py     # Логіка обчислення оцінок
└── utils/
    ├── __init__.py
    └── validators.py         # Функції валідації даних
"""
print(project_structure)

# __init__.py може реекспортувати імена для зручності
# Файл data/__init__.py:
# from .loader import load_students
#
# Тепер можна:
# from data import load_students   (замість from data.loader import load_students)


student_manager/
├── main.py                  # Точка входу
├── data/
│   ├── __init__.py          # from data import load_students
│   └── loader.py            # Завантаження даних
├── services/
│   ├── __init__.py
│   └── grade_service.py     # Логіка обчислення оцінок
└── utils/
    ├── __init__.py
    └── validators.py         # Функції валідації даних



---

# 3. Обробка помилок (Error Handling)

---

## 3.1 Типи винятків та ієрархія (Exception types and hierarchy)

Python використовує **винятки (exceptions)** для сигналізації про помилки. Всі винятки — це об'єкти, що утворюють ієрархію наслідування.

### Найпоширеніші винятки

| Виняток | Коли виникає |
|---------|-------------|
| `ValueError` | Неправильне значення (наприклад, `int("abc")`) |
| `TypeError` | Неправильний тип (наприклад, `"2" + 2`) |
| `KeyError` | Ключ не знайдено в словнику |
| `IndexError` | Індекс за межами списку |
| `FileNotFoundError` | Файл не знайдено |
| `ZeroDivisionError` | Ділення на нуль |
| `AttributeError` | Атрибут не існує |

### Ієрархія винятків

```
BaseException
├── SystemExit
├── KeyboardInterrupt
├── GeneratorExit
└── Exception
    ├── ArithmeticError
    │   ├── ZeroDivisionError
    │   └── OverflowError
    ├── LookupError
    │   ├── IndexError
    │   └── KeyError
    ├── ValueError
    ├── TypeError
    ├── OSError
    │   └── FileNotFoundError
    └── ...
```

![Exception Hierarchy](assets/diagrams/embedinv-exception-hierarchy-2.png)

*Джерело: [GeeksforGeeks — Exception Handling](https://www.geeksforgeeks.org/python-exception-handling/)*

In [90]:
# Демонстрація різних типів винятків
errors = {
    "ValueError": lambda: int("abc"),
    "TypeError": lambda: "2" + 2,
    "KeyError": lambda: {"a": 1}["b"],
    "IndexError": lambda: [1, 2, 3][10],
    "ZeroDivisionError": lambda: 1 / 0,
}

for name, trigger in errors.items():
    try:
        trigger()
    except Exception as e:
        print(f"{name:>20}: {e}")

          ValueError: invalid literal for int() with base 10: 'abc'
           TypeError: can only concatenate str (not "int") to str
            KeyError: 'b'
          IndexError: list index out of range
   ZeroDivisionError: division by zero


---

## 3.2 `try` / `except` / `else` / `finally`

```python
try:
    # Код, що може викликати виняток
except SpecificError as e:
    # Обробка конкретного винятку
except (Error1, Error2):
    # Обробка кількох винятків
else:
    # Виконується, якщо виняток НЕ виник
finally:
    # Виконується ЗАВЖДИ (очистка ресурсів)
```

![Try Except Flow](assets/diagrams/29-try-except-flow.png)

*Джерело: [Real Python — Python Exceptions](https://realpython.com/python-exceptions/)*

In [91]:
# Базовий try/except
def safe_divide(a, b):
    try:
        result = a / b
    except ZeroDivisionError:
        print("Помилка: ділення на нуль!")
        return None
    except TypeError as e:
        print(f"Помилка типу: {e}")
        return None
    else:
        print(f"{a} / {b} = {result:.2f}")
        return result
    finally:
        print("--- Операція завершена ---")

safe_divide(10, 3)
print()
safe_divide(10, 0)
print()
safe_divide("10", 3)

10 / 3 = 3.33
--- Операція завершена ---

Помилка: ділення на нуль!
--- Операція завершена ---

Помилка типу: unsupported operand type(s) for /: 'str' and 'int'
--- Операція завершена ---


In [ ]:
# Кілька except блоків + except Exception as e
def process_data(data):
    """Обробляє дані з обробкою різних помилок."""
    try:
        value = data["key"]
        number = int(value)
        result = 100 / number
        return result
    except KeyError:
        print("Ключ 'key' не знайдено в даних")
    except ValueError:
        print(f"Не вдалося перетворити '{data.get('key')}' на число")
    except ZeroDivisionError:
        print("Значення не може бути нулем")
    except Exception as e:
        print(f"Невідома помилка: {type(e).__name__}: {e}")
    return None

# Тестуємо різні сценарії
test_cases = [
    {"key": "5"},     # OK
    {"wrong": "5"},   # KeyError
    {"key": "abc"},   # ValueError
    {"key": "0"},     # ZeroDivisionError
]

for data in test_cases:
    result = process_data(data)
    print(f"  Вхід: {data} → Результат: {result}\n")

  Вхід: {'key': '5'} → Результат: 20.0

Ключ 'key' не знайдено в даних
  Вхід: {'wrong': '5'} → Результат: None

Не вдалося перетворити 'abc' на число
  Вхід: {'key': 'abc'} → Результат: None

Значення не може бути нулем
  Вхід: {'key': '0'} → Результат: None



In [94]:
# raise — виклик винятку
def set_age(age):
    if not isinstance(age, int):
        raise TypeError(f"Вік повинен бути int, отримано {type(age).__name__}")
    if age < 0 or age > 150:
        raise ValueError(f"Вік повинен бути від 0 до 150, отримано {age}")
    return age

try:
    set_age(25)
    print("set_age(25) — OK")
    set_age(-5)
except ValueError as e:
    print(f"ValueError: {e}")

# Власний клас винятку
class ValidationError(Exception):
    """Виняток для помилок валідації."""
    def __init__(self, field, message):
        self.field = field
        self.message = message
        super().__init__(f"{field}: {message}")

try:
    raise ValidationError("email", "Неправильний формат email")
except ValidationError as e:
    print(f"\nValidationError — поле: {e.field}, повідомлення: {e.message}")

set_age(25) — OK
ValueError: Вік повинен бути від 0 до 150, отримано -5

ValidationError — поле: email, повідомлення: Неправильний формат email


In [95]:
# Ланцюжок винятків (exception chaining): raise ... from ...
def fetch_user(user_id):
    try:
        # Симулюємо помилку бази даних
        data = {"users": {}}
        return data["users"][user_id]
    except KeyError as e:
        raise ValueError(f"Користувач {user_id} не знайдений") from e

try:
    fetch_user(42)
except ValueError as e:
    print(f"Помилка: {e}")
    print(f"Причина: {e.__cause__}")

Помилка: Користувач 42 не знайдений
Причина: 42


---

## 3.3 Практики коректного написання коду (Best practices)

### EAFP vs LBYL

| Підхід | Розшифровка | Стиль |
|--------|------------|-------|
| **EAFP** | Easier to Ask Forgiveness than Permission | Pythonic |
| **LBYL** | Look Before You Leap | Традиційний |

**EAFP vs LBYL — два підходи до обробки помилок:**

| | LBYL (Look Before You Leap) | EAFP (Easier to Ask Forgiveness) |
|---|---|---|
| **Підхід** | Перевірити перед дією | Спробувати, обробити помилку |
| **Стиль** | `if key in dict:` | `try: dict[key]` |
| **Мова** | Типово для C/Java | Pythonic |

```python
# LBYL (НЕ Pythonic)        # EAFP (Pythonic)
if key in my_dict:           try:
    value = my_dict[key]         value = my_dict[key]
else:                        except KeyError:
    value = default              value = default
```

### Антипатерни

```python
# ПОГАНО: "голий" except ловить ВСЕ, включаючи Ctrl+C
try:
    ...
except:  # НЕ робіть так!
    pass

# ПОГАНО: мовчазне проковтування помилок
try:
    ...
except Exception:
    pass  # Баг сховався!

# ДОБРЕ: ловити конкретні винятки
try:
    ...
except ValueError as e:
    logger.error(f"Validation failed: {e}")
    raise
```

**Найкращі практики обробки винятків:**

| Практика | Приклад |
|----------|---------|
| Ловіть конкретні винятки | `except ValueError` замість `except Exception` |
| Використовуйте `else` | Код після `try` який не потребує захисту |
| Завжди `finally` для cleanup | Закриття файлів, з'єднань |
| Не ігноруйте винятки | Ніколи порожній `except: pass` |
| Логуйте помилки | `logging.exception("...")` |

In [ ]:
# EAFP (Pythonic) vs LBYL (традиційний)
data = {"name": "Олена", "age": 20}

# LBYL — Look Before You Leap
if "grade" in data:
    grade = data["grade"]
else:
    grade = "N/A"
print(f"LBYL: grade = {grade}")

# EAFP — Easier to Ask Forgiveness than Permission
try:
    grade = data["grade"]
except KeyError:
    grade = "N/A"
print(f"EAFP: grade = {grade}")

# Ще простіше з .get()
grade = data.get("grade", "N/A")
print(f".get(): grade = {grade}")

---

## 3.4 Модуль `logging`

Замість `print()` для відлагодження — використовуйте `logging`:

| Рівень | Коли використовувати |
|--------|---------------------|
| `DEBUG` | Детальна діагностика |
| `INFO` | Підтвердження нормальної роботи |
| `WARNING` | Щось несподіване (але програма працює) |
| `ERROR` | Серйозна помилка (функціонал не працює) |
| `CRITICAL` | Критична помилка (програма може впасти) |

**logging vs print() — порівняння:**

| Аспект | `print()` | `logging` |
|--------|-----------|-----------|
| Рівні важливості | Ні | DEBUG, INFO, WARNING, ERROR, CRITICAL |
| Вимкнення | Видалити/коментувати | Змінити рівень |
| Файл | Тільки консоль | Консоль + файл + мережа |
| Формат | Простий текст | Час, модуль, рівень, повідомлення |
| Production | Ні | Так |


In [96]:
# Базове використання logging
import logging

# Налаштування (в notebook потрібен force=True для перезавантаження)
logging.basicConfig(
    level=logging.DEBUG,
    format="%(levelname)-8s | %(message)s",
    force=True
)

logger = logging.getLogger(__name__)

# Різні рівні
logger.debug("Початок обчислення (деталі для розробника)")
logger.info("Сервер запущено на порті 8080")
logger.warning("Диск заповнено на 90%")
logger.error("Не вдалося підключитись до бази даних")
logger.critical("Системна помилка — аварійне завершення!")

DEBUG    | Початок обчислення (деталі для розробника)
INFO     | Сервер запущено на порті 8080
WARNING  | Диск заповнено на 90%
ERROR    | Не вдалося підключитись до бази даних
CRITICAL | Системна помилка — аварійне завершення!


In [ ]:
# Практичний приклад: logging замість print()
import logging

logging.basicConfig(level=logging.INFO, format="%(levelname)s: %(message)s", force=True)
logger = logging.getLogger("grade_calculator")

def calculate_average(grades: list[int]) -> float:
    logger.debug(f"Вхідні дані: {grades}")
    if not grades:
        logger.warning("Порожній список оцінок!")
        return 0.0
    avg = sum(grades) / len(grades)
    logger.info(f"Середня оцінка: {avg:.1f}")
    return avg

# print() vs logging:
# print("Debug info") ← видалити перед деплоєм? Забудете!
# logger.debug("Debug info") ← просто змініть рівень на INFO

calculate_average([95, 87, 92, 78])
calculate_average([])

---

# 4. Практичні вправи

---

## Вправа 1: Функції — lambda, map/filter, сортування

Дано список товарів. Виконайте:
1. Відсортуйте товари за ціною (від дешевих до дорогих) за допомогою `sorted()` з `lambda`
2. Використайте `filter()` щоб знайти товари дорожчі за 500 грн
3. Використайте `map()` щоб створити список рядків формату "Назва: Ціна грн"


In [ ]:
# Вправа 1: Ваш код тут
products = [
    {"name": "Ноутбук", "price": 25000},
    {"name": "Миша", "price": 350},
    {"name": "Клавіатура", "price": 800},
    {"name": "Монітор", "price": 12000},
    {"name": "USB-кабель", "price": 120},
    {"name": "Навушники", "price": 1500},
]

# 1. Сортування за ціною
# sorted_products = ...

# 2. Товари дорожчі за 500 грн
# expensive = ...

# 3. Список рядків "Назва: Ціна грн"
# descriptions = ...


<details>
<summary>Рішення (клікніть щоб побачити)</summary>

```python
products = [
    {"name": "Ноутбук", "price": 25000},
    {"name": "Миша", "price": 350},
    {"name": "Клавіатура", "price": 800},
    {"name": "Монітор", "price": 12000},
    {"name": "USB-кабель", "price": 120},
    {"name": "Навушники", "price": 1500},
]

# 1. Сортування за ціною
sorted_products = sorted(products, key=lambda p: p["price"])
print("За ціною:")
for p in sorted_products:
    print(f"  {p['name']}: {p['price']} грн")

# 2. Товари дорожчі за 500 грн
expensive = list(filter(lambda p: p["price"] > 500, products))
print(f"\nДорожчі за 500: {[p['name'] for p in expensive]}")

# 3. Список рядків
descriptions = list(map(lambda p: f"{p['name']}: {p['price']} грн", products))
print(f"\nОписи: {descriptions}")
```
</details>

---

## Вправа 2: Модулі — створення модуля валідаторів

Напишіть набір функцій-валідаторів:
1. `validate_email(email)` — перевіряє наявність `@` та `.` після `@`
2. `validate_age(age)` — вік від 1 до 150
3. `validate_password(password)` — мінімум 8 символів, хоча б 1 цифра


In [ ]:
# Вправа 2: Ваш код тут

# def validate_email(email: str) -> bool:
#     ...

# def validate_age(age: int) -> bool:
#     ...

# def validate_password(password: str) -> bool:
#     ...

# Тест:
# print(validate_email("user@example.com"))  # True
# print(validate_email("invalid"))           # False
# print(validate_age(25))                    # True
# print(validate_password("abc123xy"))       # True
# print(validate_password("short"))          # False


<details>
<summary>Рішення (клікніть щоб побачити)</summary>

```python
def validate_email(email: str) -> bool:
    if "@" not in email:
        return False
    domain = email.split("@")[-1]
    return "." in domain

def validate_age(age: int) -> bool:
    return isinstance(age, int) and 1 <= age <= 150

def validate_password(password: str) -> bool:
    return len(password) >= 8 and any(c.isdigit() for c in password)

# Тести
tests = [
    ("validate_email('user@example.com')", validate_email("user@example.com")),
    ("validate_email('invalid')", validate_email("invalid")),
    ("validate_age(25)", validate_age(25)),
    ("validate_age(-5)", validate_age(-5)),
    ("validate_password('abc123xy')", validate_password("abc123xy")),
    ("validate_password('short')", validate_password("short")),
]

for desc, result in tests:
    status = "PASS" if result == (not "invalid" in desc and not "-5" in desc and not "short" in desc) else "CHECK"
    print(f"  {desc} → {result}")
```
</details>

---

## Вправа 3: Помилки — безпечний обробник даних

Напишіть функцію `safe_process(data_list)`, яка:
1. Приймає список словників `[{"value": "123"}, {"value": "abc"}, ...]`
2. Конвертує `value` в `int` і ділить 1000 на нього
3. Обробляє `ValueError` (не число), `ZeroDivisionError`, `KeyError` (немає ключа)
4. Повертає список результатів, замінюючи помилки на `None`


In [ ]:
# Вправа 3: Ваш код тут

# def safe_process(data_list: list[dict]) -> list[float | None]:
#     ...

# Тест:
# test_data = [
#     {"value": "5"},      # 1000/5 = 200.0
#     {"value": "abc"},    # ValueError → None
#     {"value": "0"},      # ZeroDivisionError → None
#     {"wrong_key": "10"}, # KeyError → None
#     {"value": "4"},      # 1000/4 = 250.0
# ]
# results = safe_process(test_data)
# print(results)  # [200.0, None, None, None, 250.0]


<details>
<summary>Рішення (клікніть щоб побачити)</summary>

```python
def safe_process(data_list: list[dict]) -> list[float | None]:
    results = []
    for item in data_list:
        try:
            value = int(item["value"])
            result = 1000 / value
            results.append(result)
        except KeyError:
            print(f"  KeyError: ключ 'value' не знайдено в {item}")
            results.append(None)
        except ValueError:
            print(f"  ValueError: '{item['value']}' не є числом")
            results.append(None)
        except ZeroDivisionError:
            print(f"  ZeroDivisionError: ділення на нуль")
            results.append(None)
    return results

test_data = [
    {"value": "5"},
    {"value": "abc"},
    {"value": "0"},
    {"wrong_key": "10"},
    {"value": "4"},
]

results = safe_process(test_data)
print(f"\nРезультати: {results}")
```
</details>

---

# 6. Міні-проєкт: Конвеєр обробки даних студентів (Student Data Processing Pipeline)

**Завдання**: Створити конвеєр обробки даних студентів, що інтегрує:
- **Функції**: lambda, map, filter, sorted
- **Модулі**: власні функції валідації
- **Обробка помилок**: валідація вхідних даних з try/except

### Вимоги:
1. Функції валідації для перевірки даних студентів (ім'я, вік, оцінки)
2. Обробка даних: фільтрація відмінників, сортування за середньою оцінкою, форматований звіт
3. Обробка помилок для невалідних даних з використанням власного винятку `ValidationError`
4. Фінальний конвеєр, що об'єднує всі кроки

In [ ]:
# Крок 1: Визначте функції валідації даних
# Ваш код тут

# class ValidationError(Exception):
#     """Виняток для помилок валідації."""
#     pass

# def validate_name(name: str) -> str:
#     """Валідує та очищає ім'я студента."""
#     ...

# def validate_age(age) -> int:
#     """Валідує вік (16-100)."""
#     ...

# def validate_grades(grades) -> list[int]:
#     """Валідує список оцінок (0-100)."""
#     ...

In [ ]:
# Крок 2: Створіть функції обробки даних
# Ваш код тут

# def create_student(name: str, age: int, grades: list[int]) -> dict:
#     """Створює запис студента як словник."""
#     ...

# def average_grade(student: dict) -> float:
#     """Обчислює середню оцінку студента."""
#     ...

# def is_excellent(student: dict) -> bool:
#     """Перевіряє чи студент є відмінником (середня >= 90)."""
#     ...

# def get_excellent_students(students: list[dict]) -> list[dict]:
#     """Фільтрує відмінників за допомогою filter + lambda."""
#     ...

# def sort_by_average(students: list[dict], reverse: bool = True) -> list[dict]:
#     """Сортує студентів за середньою оцінкою."""
#     ...

In [ ]:
# Крок 3: Додайте обробку помилок
# Ваш код тут

# def add_student_safe(students: list, name, age, grades) -> None:
#     """Безпечно додає студента з валідацією та обробкою помилок."""
#     try:
#         validated_name = validate_name(name)
#         validated_age = validate_age(age)
#         validated_grades = validate_grades(grades)
#         student = create_student(validated_name, validated_age, validated_grades)
#         students.append(student)
#         print(f"  Додано: {student['name']} (середня: {average_grade(student):.1f})")
#     except ValidationError as e:
#         print(f"  Помилка валідації для '{name}': {e}")
#     except Exception as e:
#         print(f"  Неочікувана помилка: {e}")

In [ ]:
# Крок 4: Зберіть все разом — конвеєр обробки
# Ваш код тут

<details>
<summary>Повне рішення (клікніть щоб побачити)</summary>

```python
class ValidationError(Exception):
    """Виняток для помилок валідації."""
    pass


# Крок 1: Функції валідації
def validate_name(name: str) -> str:
    if not isinstance(name, str) or not name.strip():
        raise ValidationError("Ім'я не може бути порожнім")
    return name.strip()


def validate_age(age) -> int:
    if not isinstance(age, int) or not (16 <= age <= 100):
        raise ValidationError(f"Вік повинен бути від 16 до 100, отримано: {age}")
    return age


def validate_grades(grades) -> list[int]:
    if not isinstance(grades, (list, tuple)):
        raise ValidationError("Оцінки повинні бути списком")
    result = []
    for g in grades:
        if not isinstance(g, (int, float)) or not (0 <= g <= 100):
            raise ValidationError(f"Оцінка повинна бути від 0 до 100, отримано: {g}")
        result.append(int(g))
    return result


# Крок 2: Функції обробки даних
def create_student(name: str, age: int, grades: list[int]) -> dict:
    return {
        "name": validate_name(name),
        "age": validate_age(age),
        "grades": validate_grades(grades),
    }


def average_grade(student: dict) -> float:
    grades = student["grades"]
    return sum(grades) / len(grades) if grades else 0.0


def is_excellent(student: dict) -> bool:
    return average_grade(student) >= 90


def get_excellent_students(students: list[dict]) -> list[dict]:
    return list(filter(is_excellent, students))


def sort_by_average(students: list[dict], reverse: bool = True) -> list[dict]:
    return sorted(students, key=lambda s: average_grade(s), reverse=reverse)


def generate_report(students: list[dict]) -> str:
    sorted_students = sort_by_average(students)
    lines = [
        "=== Звіт: Конвеєр обробки даних студентів ===",
        f"Всього студентів: {len(students)}",
    ]
    for i, s in enumerate(sorted_students, 1):
        avg = average_grade(s)
        status = "★" if is_excellent(s) else " "
        lines.append(f"  {i}. [{status}] {s['name']} (вік {s['age']}) — середня: {avg:.1f}")

    excellent = get_excellent_students(students)
    lines.append(f"\nВідмінників: {len(excellent)} з {len(students)}")
    avg_all = sum(map(lambda s: average_grade(s), students)) / len(students) if students else 0
    lines.append(f"Загальна середня: {avg_all:.1f}")
    return "\n".join(lines)


# Крок 3: Безпечне додавання з обробкою помилок
def add_student_safe(students: list, name, age, grades) -> None:
    try:
        student = create_student(name, age, grades)
        students.append(student)
        print(f"  Додано: {student['name']} (середня: {average_grade(student):.1f})")
    except ValidationError as e:
        print(f"  Помилка валідації для '{name}': {e}")
    except Exception as e:
        print(f"  Неочікувана помилка: {e}")


# Крок 4: Конвеєр
print("--- Додаємо студентів ---")
students = []
add_student_safe(students, "Олена", 20, [95, 92, 98])
add_student_safe(students, "Ігор", 21, [87, 78, 90])
add_student_safe(students, "Марія", 19, [92, 95, 88])
add_student_safe(students, "Андрій", 22, [65, 70, 72])
add_student_safe(students, "", 20, [90])           # Помилка: порожнє ім'я
add_student_safe(students, "Тест", 200, [90])      # Помилка: вік
add_student_safe(students, "Тест", 20, [150])      # Помилка: оцінка

print()
print(generate_report(students))
```
</details>

---

# Підсумок (Summary)

### Що ми вивчили сьогодні:

- **Функції**: lambda-вирази, функції як об'єкти першого класу, map/filter/reduce, генератори та ітератори, правило LEGB та замикання, декоратори, type hints, docstrings

- **Модулі та імпорти**: різні стилі імпорту, стандартна бібліотека, створення власних модулів, `if __name__ == "__main__"`, структура пакетів

- **Обробка помилок**: ієрархія винятків, `try`/`except`/`else`/`finally`, `raise` та власні винятки, EAFP vs LBYL

- **Логування**: модуль `logging` як альтернатива `print()`

---

## Що далі?
### Лекція 5: ООП в Python (об'єктно-орієнтоване програмування) та Робота з файлами

У наступній лекції ми повністю зануримось в **ООП (об'єктно-орієнтоване програмування)**:
- **Основи ООП**: класи, об'єкти, `__init__`, `self`, атрибути та методи
- **4 принципи ООП**: інкапсуляція, наслідування, поліморфізм, абстракція
- **Просунуті можливості**: `@dataclass`, магічні методи (`__repr__`, `__str__`, `__eq__`), композиція vs наслідування
- **Python ООП vs Java/C#/C++**: порівняння підходів у різних мовах
- **Робота з файлами**: читання/запис текстових файлів, контекстний менеджер `with`
- **JSON та CSV**: серіалізація/десеріалізація даних

---

## Домашнє завдання

1. **Функції**: Напишіть генератор `prime_numbers()`, що генерує нескінченну послідовність простих чисел. Візьміть перші 20.
2. **Модулі**: Створіть модуль `utils/math_utils.py` з функціями `factorial()`, `fibonacci(n)`, `is_prime(n)`. Додайте `if __name__ == "__main__":` з тестами.
3. **Помилки**: Напишіть функцію `safe_json_parse(json_string)`, що парсить JSON з обробкою всіх можливих помилок.

---

# Джерела (References)

## Офіційна документація

- [Functions](https://docs.python.org/3/tutorial/controlflow.html#more-on-defining-functions) — визначення функцій
- [Lambda Expressions](https://docs.python.org/3/reference/expressions.html#lambda) — lambda-вирази
- [Modules](https://docs.python.org/3/tutorial/modules.html) — модулі та пакети
- [Errors and Exceptions](https://docs.python.org/3/tutorial/errors.html) — обробка помилок
- [Built-in Exceptions](https://docs.python.org/3/library/exceptions.html) — ієрархія винятків
- [logging](https://docs.python.org/3/library/logging.html) — модуль логування
- [typing](https://docs.python.org/3/library/typing.html) — підказки типів

## Туторіали

- [Real Python — Lambda Functions](https://realpython.com/python-lambda/)
- [Real Python — Python Scope & LEGB Rule](https://realpython.com/python-scope-legb-rule/)
- [Medium - Python Generators Explained for Beginners](https://medium.com/cloud-for-everybody/python-generators-explained-for-beginners-cb5d777147fb)
- [Real Python - Iterators and Iterables in Python](https://realpython.com/python-iterators-iterables/)
- [Real Python — Primer on Decorators](https://realpython.com/primer-on-python-decorators/)
- [Real Python — Python Modules and Packages](https://realpython.com/python-modules-packages/)
- [Real Python — Python Exceptions](https://realpython.com/python-exceptions/)
- [Real Python — Python Type Checking](https://realpython.com/python-type-checking/)
- [Real Python — Python Logging](https://realpython.com/python-logging/)


## Поглиблене вивчення

- [PEP 257 — Docstring Conventions](https://peps.python.org/pep-0257/)
- [PEP 484 — Type Hints](https://peps.python.org/pep-0484/)
- [GeeksforGeeks — Exception Handling](https://www.geeksforgeeks.org/python-exception-handling/)